In [1]:
import pandas as pd
import spacy
from spacy import displacy  # Entity Visualizer
from collections import Counter

In [2]:
import en_core_web_lg
nlp = en_core_web_lg.load()
from bs4 import BeautifulSoup
import requests
import re

In [3]:
def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib') #Python library for pulling data out of HTML and XML files.
    for script in soup(["script", "style", 'aside']):  #Remove all the java script and style sheet codes
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))


In [4]:
ny_bb = url_to_string('https://www.marketwatch.com/story/big-week-lifts-apple-to-dows-top-performing-stock-2018-09-01')
article = nlp(ny_bb)


In [5]:
len(article.ents)

184

In [6]:
labels = [x.label_ for x in article.ents]
Counter(labels)


Counter({'ORG': 89,
         'WORK_OF_ART': 12,
         'DATE': 24,
         'GPE': 9,
         'LOC': 2,
         'PERSON': 7,
         'CARDINAL': 8,
         'TIME': 3,
         'PERCENT': 15,
         'ORDINAL': 2,
         'PRODUCT': 3,
         'QUANTITY': 2,
         'EVENT': 1,
         'MONEY': 7})

In [7]:
## Most common tokens ####
items = [x.text for x in article.ents]
Counter(items).most_common(3)


[('Apple', 12), ("Barron's", 4), ('U.S.', 3)]

In [12]:
## Randomly selecting sentence #####
sentences = [x for x in article.sents]
print(sentences[55])

Apple stays in the tech sector, but Amazon is being reassigned to the consumer discretionary sector, and both Facebook and Alphabet are being added to a new communications services sector that is replacing the old telecom sector.


In [14]:
displacy.render(nlp(str(sentences[55])), jupyter=True, style='ent')

In [15]:
displacy.render(nlp(str(sentences[58])), style='dep', jupyter = False)
print([(x, x.ent_type_) for x in sentences[55]])

[(Apple, 'ORG'), (stays, ''), (in, ''), (the, ''), (tech, ''), (sector, ''), (,, ''), (but, ''), (Amazon, 'ORG'), (is, ''), (being, ''), (reassigned, ''), (to, ''), (the, ''), (consumer, ''), (discretionary, ''), (sector, ''), (,, ''), (and, ''), (both, ''), (Facebook, 'ORG'), (and, ''), (Alphabet, 'PERSON'), (are, ''), (being, ''), (added, ''), (to, ''), (a, ''), (new, ''), (communications, ''), (services, ''), (sector, ''), (that, ''), (is, ''), (replacing, ''), (the, ''), (old, ''), (telecom, ''), (sector, ''), (., '')]


In [16]:
## for entire article ####
displacy.render(article, jupyter=True, style='ent')

In [17]:
##Redaction Replacements ####
def replace_name_with_placeholder(token):
    if token.ent_iob != 0 and token.ent_type_ == "PERSON":
        return "[SANITIZED_PERSON] "
    elif token.ent_iob != 0 and token.ent_type_ == "DATE":
        return "[SANITIZED_DATE] "
    elif token.ent_iob != 0 and token.ent_type_ == "GPE":
        return "[SANITIZED_GPE] "
    else:
        return token.string
    
def scrub(text):
    doc = nlp(text)
    for ent in doc.ents:
        ent.merge()
    tokens = map(replace_name_with_placeholder, doc)
    return "".join(tokens)

print(scrub(ny_bb))


   Big Week Lifts Apple to Dow’s Top-Performing Stock - MarketWatch                                                                                                                                           MarketWatch site logo                                                                                                                                                                                                                                                                                                                                                                                       Latest             Watchlist             Markets             Investing             Barron's             Economy             Personal Finance             Retirement             Video             Opinion                                                                     Sign Up                                  •                                      Log In                                          